In [1]:
from premise import *
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
#bw2data.projects.set_current("ecoinvent-3.8-cutoff")
#bw2data.projects.set_current("ecoinvent-3.9.1-cutoff_bw25")
#clear_inventory_cache()
scenarios = [
    {"model": "remind", "pathway": "SSP2-rollBack", "year": 2050},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2050},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2050},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.10-cutoff",
        source_version="3.10",
        biosphere_name="ecoinvent-3.10-biosphere",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        #system_model="consequential",
        keep_imports_uncertainty=True,
        keep_source_db_uncertainty=True
)
#ndb.write_db_to_brightway("test final energy 2")

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [2]:
ndb.update()

Processing scenarios for all sectors:   0%|     | 0/1 [00:00<?, ?it/s]

Processing scenarios for all sectors:   0%|     | 0/1 [01:25<?, ?it/s]


KeyError: "The variable(s) ['truck, battery electric, 3.5 metric ton', 'truck, fuel cell electric, 3.5 metric ton', 'truck, diesel, 3.5 metric ton', 'truck, compressed gas, 3.5 metric ton', 'truck, battery electric, 7.5 metric ton', 'truck, fuel cell electric, 7.5 metric ton', 'truck, diesel, 7.5 metric ton', 'truck, compressed gas, 7.5 metric ton', 'truck, battery electric, 18 metric ton', 'truck, fuel cell electric, 18 metric ton', 'truck, diesel, 18 metric ton', 'truck, compressed gas, 18 metric ton', 'truck, battery electric, 26 metric ton', 'truck, fuel cell electric, 26 metric ton', 'truck, diesel, 26 metric ton', 'truck, compressed gas, 26 metric ton', 'truck, battery electric, 40 metric ton', 'truck, fuel cell electric, 40 metric ton', 'truck, diesel, 40 metric ton', 'truck, compressed gas, 40 metric ton'] not found in production volumes data. Available variables: []."

In [3]:
ndb.write_db_to_brightway()

In [4]:
ndb.generate_change_report()

'CAZ'

In [3]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Title: Writing activities to SQLite3 database:
  Started: 05/31/2025 21:25:58
  Finished: 05/31/2025 21:26:18
  Total time elapsed: 00:00:19
  CPU %: 92.40
  Memory %: 15.12
Created database: test cng 3
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [ ]:
from premise.activity_maps import InventorySet, debug_mapping_to_dataframe
from premise.utils import load_database

for scenario in ndb.scenarios:
    scenario = load_database(scenario)

In [16]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["type"] == "technosphere":
            if e["name"] == "primary zinc production from concentrate" and e["product"] == "cadmium":
                print(ds["name"], ds["location"], e["name"], e["product"], e["location"])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 335965 stored elements and shape (2687, 26329)>

In [3]:
    df = debug_mapping_to_dataframe(
    scenarios[0]
)

In [4]:
df.to_excel("mapping.xlsx")

In [5]:
inv = InventorySet(
    database=ndb.scenarios[0]["database"],
    version="3.10",
    model="remind"
)
biomass_map = inv.generate_cement_map()
df_biomass = debug_mapping_to_dataframe(biomass_map)
df_biomass

In [5]:
dataset = bw2data.Database("ecoinvent-3.11-cutoff").random()

list = []

for exc in dataset.exchanges():
    list.append(
        (
            dataset["name"],
            dataset.get("comment", ""),
            exc["name"],
            exc.input.get("reference product"),
            exc.input.get("location"),
            exc["type"],
            exc["amount"],
            exc["unit"],
        )
    )

,Category,Market,Product,Locations
0,"cement, dry feed rotary kiln",clinker production,clinker,"[BR, CA-QC, CAZ, CH, CHA, CO, EC, EUR, Europe ..."
1,"cement, dry feed rotary kiln, efficient, with ...",clinker production,clinker,"[BR, CA-QC, CAZ, CH, CHA, CO, EC, EUR, Europe ..."
2,,"clinker production, dry feed rotary kiln, effi...",clinker,"[CAZ, CHA, EUR, Europe without Switzerland, IN..."


In [ ]:
import pandas as pd
pd.DataFrame(list, columns=["dataset", "comment","name", "product", "location", "type", "amount", "unit"])

In [9]:
for ds in ndb.scenarios[3]["database"]:
    if ds["name"] == "transport, passenger bus, gasoline, 13m single deck urban bus, EURO-VI":
        print(ds["location"])

In [6]:
t = [
    tech for tech in ndb.scenarios[0]["iam data"].daccs_energy_use.variables.values
]

market for natural gas, low pressure JPN natural gas, low pressure
market for natural gas, low pressure 1.0 JPN production
biomethane production, from biogas upgrading, using amine scrubbing 0.25094126690230667 JPN technosphere
natural gas, high pressure, import from AE 0.006514966900189843 RoW technosphere
natural gas, high pressure, import from AU 0.13096181401672782 RoW technosphere
natural gas, high pressure, import from EG 0.0009565722182054117 RoW technosphere
natural gas, high pressure, import from ID 0.009271498155888725 RoW technosphere
natural gas, high pressure, import from MY 0.050056608013976156 RoW technosphere
natural gas, high pressure, import from NG 0.004153829905826414 RoW technosphere
natural gas, high pressure, import from OM 0.009361828196270407 RoW technosphere
natural gas, high pressure, import from PE 0.002624487359181432 JPN technosphere
natural gas, high pressure, import from QA 0.044482293487315694 RoW technosphere
natural gas, high pressure, import from RU 

In [7]:
[x for x in t if x not in ndb.scenarios[0]["iam data"].production_volumes.variables.values]

market for transport, passenger car, EURO 3 1.0 RER production
transport, passenger car, EURO 3 1.0 RER technosphere

market for transport, passenger car with internal combustion engine 1.0 RER production
transport, passenger car with internal combustion engine 1.0 RER technosphere

market for transport, passenger car, EURO 4 1.0 RER production
transport, passenger car, EURO 4 1.0 RER technosphere

market for transport, passenger car, EURO 5 1.0 RER production
transport, passenger car, EURO 5 1.0 RER technosphere

market for transport, passenger car 1.0 RER production
market for transport, passenger car 1.0 EUR technosphere



In [5]:
ndb.scenarios[0]["iam data"].production_volumes.sel(
    year=2060, variables=t
)

market for sawlog and veneer log, softwood, measured as solid wood under bark production CHA 1.0
market for transport, freight train technosphere CN 13.2253954116397
market for transport, freight, lorry, unspecified technosphere RoW 5.805904223724
market for transport, freight, sea, bulk carrier for dry goods technosphere GLO 15.6522262861392
softwood forestry, pine, sustainable forest management technosphere RoW 0.448540479487862
softwood forestry, spruce, sustainable forest management technosphere RoW 0.551459520512138



In [ ]:
 ndb.scenarios[0]["iam data"].daccs_energy_use.sel(region="CAZ", year=2060)

In [ ]:
    ei311 = list(set([
    (
        act["name"],
        act["reference product"],
    )
    for act in bw2data.Database("ecoinvent-3.11-cutoff")
]))

In [ ]:
ndb.write_db_to_simapro()

In [ ]:
ndb.scenarios[1]["iam data"].carbon_capture_rate

In [ ]:
from collections import defaultdict
res = defaultdict(float)
for ds in ndb.scenarios[1]["database"]:
    if ds["name"]=="market group for heat, district or industrial, natural gas":
        print(ds["location"], ds["name"])
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"), e["type"])
        print()


In [ ]:
from premise.geomap import Geomap
geo = Geomap("remind")
geo.ecoinvent_to_iam_location("Europe, without Russia and Turkey")

In [ ]:
geo.rev_additional_mappings

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame.from_dict(res, orient="index").plot(kind="bar", figsize=(20, 5))
plt.ylabel("kg heavy fuel oil/kWh")
plt.savefig("electricity production, oil.png")

In [ ]:
ndb.scenarios[0]["iam data"].electricity_efficiencies.sel(variables="Biomass IGCC CCS", year=2050)

In [ ]:
ndb.scenarios[0]["iam data"].gas_markets.sel(region="World")

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"] == "market for platinum" and e["product"] == "platinum":
            print(ds["name"], ds["location"], e["name"], e["location"])

In [ ]:
from premise.activity_maps import InventorySet
inv = InventorySet(ndb.database, "3.9")

In [ ]:
inv.generate_powerplant_map()

In [ ]:
ndb.write_db_to_brightway("test_metals")

In [ ]:
len(acts)

In [ ]:
import bw2data
bw2io.BW2Package.export_obj(obj=bw.Database("brian"), filename="db_for_brian.bw2package")

In [ ]:
ei_acts = list(set([(a["name"], a["reference product"]) for a in bw.Database("ecoinvent 3.9.1 cutoff")]))
print(len(ei_acts))

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) not in ei_acts:
        print(ds["name"])

In [ ]:
for ds in i.data:
    print(ds["name"])

In [ ]:
i.statistics()

In [ ]:
bw.get_activity(('brian', '360c5e612a154ebb8cee2650bcc1bc75_copy1'))

In [ ]:
bw.projects.set_current("ei39")

In [ ]:
import csv

with open("flows_biosphere_39.csv", "w", newline="", ) as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(l)

In [ ]:
import numpy as np
for ds in ndb.scenarios[1]["database"]:
    if ds["name"] == ("carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with waste heat, and grid electricity",
                            "carbon dioxide, captured from atmosphere"):
            print(ds["name"], ds["location"])

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU", year=2050)

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.update_all()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.write_db_to_brightway(["test_dac", "test_dac2"])

In [ ]:
Geomap()

In [ ]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="steel")